# U.S. Medical Insurance Costs

In [1]:
import pandas as pd
import csv

In [2]:
dataset = {}
with open("insurance.csv", newline='') as insurance_csv:
    dict_reader = csv.DictReader(insurance_csv) # converts the lines of our CSV file to Python dictionaries
    test = dict_reader
    column_names = dict_reader.fieldnames
    for column_name in column_names:
        dataset[column_name] = []
    for row in dict_reader:
        # row is an ordered dictionary entry containing a tuple with the column heading and the data for each observation
        # key is the column heading and the value is data at each observation
        for key, value in row.items():
            dataset[key].append(value)

In [8]:
dataset.keys()

dict_keys(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'])

In [11]:
for key in dataset.keys():
    print(len(dataset[key]))

1338
1338
1338
1338
1338
1338
1338
